In [6]:
import pandas as pd
import praw # Python Reddit API Wrapper
import re
import numpy as np
import datetime

In [2]:
# Configuration
CLIENT_ID = 'b3CUxOmuvB9QeRasUu3Hew'
CLIENT_SECRET = 'CqqnRJucVB500qgPSWTH9wHHx2acQQ'
USER_AGENT = 'script:my_reddit_script:v1.0 (by /u/py_dev684)'

# Initialize the Reddit client
reddit = praw.Reddit(
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    user_agent=USER_AGENT
)

In [7]:
def fetch_comments_with_phrases(subreddits, patterns, comment_target=20):
    """Fetch comments that contain a target phrase, sourced from 'new' posts from the specified subreddits. Comments are sourced from new posts in each subreddit. The new sorting method gives posts in the order they were posted."""

    data = []  # List to store data before converting to DataFrame

    for subreddit_name in subreddits:
        print(f"Fetching from r/{subreddit_name}...")
        subreddit = reddit.subreddit(subreddit_name)

        comment_counter = 0
        while len(data) < comment_target:
            for post in subreddit.new(limit=100):  # We'll retrieve posts in chunks of 100
                post.comments.replace_more(limit=None)  # Replace "MoreComments" with actual comments
                
                # Filter comments based on presence of target phrases
                for comment in post.comments:
                    comment_counter += 1
                    matched_pattern = next((pattern for pattern in patterns if re.search(pattern, comment.body, re.IGNORECASE)), None)
                    if matched_pattern:
                        comment_date = datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                        data.append({
                            'Subreddit': subreddit_name,
                            'Post Title': post.title,
                            'Comment ID': comment.id,
                            'Comment Date': comment_date,
                            'Comment Author': str(comment.author),
                            'Comment': comment.body,
                            'Matched Phrase': matched_pattern, 
                            'Upvotes': comment.score
                            
                        })

                    if len(data) >= comment_target:  # If we've hit our comment target, break out
                        break

                print(f"Comments checked: {comment_counter}")
                print(f"Relevant comments: {len(data)}")
                
                if len(data) >= comment_target:  # If we've hit our comment target, stop processing
                    break
    print(f"Total comments checked: {comment_counter}")          
    print(f"{len(data)} comments collected.")

    # Convert the data list into a DataFrame
    df = pd.DataFrame(data)

    return df


In [8]:
subreddits = ['finance']
patterns = [r'\bHSBC\b', r'\bCiti\b', r'\bNatWest\b', r'\bCoutts\b', r'\bLloyds\b', r'\bBarclays\b', r'\bStandard\s+Chartered\b', r'\bSantander\b', r'\bBank\s+of\s+England\b', r'\bBoE\b', r'\bGoldman\s+Sachs\b', r'\bMorgan\s+Stanley\b', r'\bSilicon\s+Valley\s+Bank\b', r'\bSVB\b', r'\bCredit\s+Suisse\b']

df = fetch_comments_with_phrases(subreddits, patterns)

Fetching from r/finance...
Comments checked: 30
Relevant comments: 0
Comments checked: 38
Relevant comments: 0
Comments checked: 68
Relevant comments: 0
Comments checked: 71
Relevant comments: 0
Comments checked: 87
Relevant comments: 0
Comments checked: 98
Relevant comments: 0
Comments checked: 114
Relevant comments: 0
Comments checked: 129
Relevant comments: 2
Comments checked: 138
Relevant comments: 2
Comments checked: 150
Relevant comments: 2
Comments checked: 170
Relevant comments: 2
Comments checked: 188
Relevant comments: 3
Comments checked: 192
Relevant comments: 3
Comments checked: 201
Relevant comments: 3
Comments checked: 201
Relevant comments: 3
Comments checked: 218
Relevant comments: 3
Comments checked: 249
Relevant comments: 3
Comments checked: 254
Relevant comments: 3
Comments checked: 271
Relevant comments: 3
Comments checked: 296
Relevant comments: 3
Comments checked: 320
Relevant comments: 3
Comments checked: 337
Relevant comments: 3
Comments checked: 352
Relevant co

In [14]:
import praw
import datetime
import re

def fetch_comments_from_time_period(subreddits, patterns, time_period: int=24):
    """Fetch all comments from the specified subreddits that fall within the given time period."""

    # Define the end time of our search (current time)
    end_time = datetime.datetime.utcnow()
    start_time = end_time - datetime.timedelta(hours=time_period)

    # Convert start and end times to Unix timestamps
    start_timestamp = start_time.timestamp()
    end_timestamp = end_time.timestamp()

    data = []  # List to store data before converting to DataFrame
    comment_counter = 0
    
    for subreddit_name in subreddits:
        print(f"Fetching from r/{subreddit_name}...")
        subreddit = reddit.subreddit(subreddit_name)


        for comment in subreddit.stream.comments(skip_existing=True):
            comment_counter += 1
            # Check if the comment is within our time range
            if start_timestamp <= comment.created_utc <= end_timestamp:
                # Check if the comment matches any of the patterns
                matched_pattern = next((pattern for pattern in patterns if re.search(pattern, comment.body, re.IGNORECASE)), None)
                if matched_pattern:
                    comment_date = datetime.datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S')
                    data.append({
                        'comment_id': comment.id,
                        'comment_date': comment_date,
                        'subreddit': subreddit_name,
                        'comment_author': str(comment.author),
                        'comment': comment.body,
                        'matched_phrase': matched_pattern,
                        'upvotes': comment.score
                    })

            print(f"Comments checked: {comment_counter}")
            print(f"Relevant comments: {len(data)}")

    # Convert the data list into a DataFrame
    df = pd.DataFrame(data)

    return df


In [15]:
subreddits = ['finance']
patterns = [r'\bHSBC\b', r'\bCiti\b', r'\bNatWest\b', r'\bCoutts\b', r'\bLloyds\b', r'\bBarclays\b', r'\bStandard\s+Chartered\b', r'\bSantander\b', r'\bBank\s+of\s+England\b', r'\bBoE\b', r'\bGoldman\s+Sachs\b', r'\bMorgan\s+Stanley\b', r'\bSilicon\s+Valley\s+Bank\b', r'\bSVB\b', r'\bCredit\s+Suisse\b']

df = fetch_comments_from_time_period(subreddits, patterns, time_period = 24)

Fetching from r/finance...


KeyboardInterrupt: 

In [9]:
df

,Subreddit,Post Title,Comment ID,Comment Date,Comment Author,Comment,Matched Phrase,Upvotes
0,finance,Morale at Goldman Sachs 'is pretty good': CEO ...,ketusqi,2023-12-25 04:35:35,Fack_JeffB_n_KenG,Fuck Goldman Sachs.,\bGoldman\s+Sachs\b,22
1,finance,Morale at Goldman Sachs 'is pretty good': CEO ...,kerzsb5,2023-12-24 19:43:39,Bartinhoooo,"‚Fuck sustainability, that company pays 10% in...",\bGoldman\s+Sachs\b,-13
2,finance,Why Wall Street Is Suddenly Having an Everythi...,kdgd779,2023-12-15 12:26:47,savagepanda,It’s odd how fed suddenly became dovish. They...,\bSVB\b,12
3,finance,I'm a reporter at Bloomberg News. We just publ...,k9dvmve,2023-11-15 17:48:57,jigsaw_faust,I work for a credit union and it’s been crazy ...,\bSantander\b,23
4,finance,Federal Reserve holds interest rates at highes...,k8gbthi,2023-11-09 03:10:19,TowerJanitor,“Oops we were wrong oops”\n\nEvery single bulg...,\bCiti\b,1
5,finance,Morale at Goldman Sachs 'is pretty good': CEO ...,ketusqi,2023-12-25 04:35:35,Fack_JeffB_n_KenG,Fuck Goldman Sachs.,\bGoldman\s+Sachs\b,19
6,finance,Morale at Goldman Sachs 'is pretty good': CEO ...,kerzsb5,2023-12-24 19:43:39,Bartinhoooo,"‚Fuck sustainability, that company pays 10% in...",\bGoldman\s+Sachs\b,-14
7,finance,Why Wall Street Is Suddenly Having an Everythi...,kdgd779,2023-12-15 12:26:47,savagepanda,It’s odd how fed suddenly became dovish. They...,\bSVB\b,13
8,finance,I'm a reporter at Bloomberg News. We just publ...,k9dvmve,2023-11-15 17:48:57,jigsaw_faust,I work for a credit union and it’s been crazy ...,\bSantander\b,21
9,finance,Federal Reserve holds interest rates at highes...,k8gbthi,2023-11-09 03:10:19,TowerJanitor,“Oops we were wrong oops”\n\nEvery single bulg...,\bCiti\b,1


# Sentiment Analysis

## NLTK Vader

In [56]:
# Load sentiment analyser
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

# Initialise VADER
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/jackwalker/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [57]:
# Apply VADER analysis on text column
df['sentiment_scores'] = df['Comment'].apply(lambda x: sia.polarity_scores(x))
df['compound'] = df['sentiment_scores'].apply(lambda score_dict: score_dict['compound'])
df['sentiment'] = df['compound'].apply(lambda c: 'POSITIVE' if c >= 0.05 else ('NEGATIVE' if c <= -0.05 else 'NEUTRAL'))
df.head(20)

,Subreddit,Post Title,Comment Author,Comment,Matched Phrase,Upvotes,sentiment_scores,compound,sentiment
0,finance,‘Almost All Loans Are Bad’—Why Banks Aren’t Le...,hcbaron,Extracted article:\n\nBanks would love to lend...,\bBarclays\b,13,"{'neg': 0.049, 'neu': 0.799, 'pos': 0.152, 'co...",0.9977,POSITIVE
1,finance,"Moronic Monday - September 05, 2023 - Your Wee...",14446368,Could we un-pin the SVB thing now?,\bSVB\b,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL
2,finance,Bloomberg overhauls management team with Mark ...,marketrent,Per an internal memo sent by founder Mike Bloo...,\bBank\s+of\s+England\b,2,"{'neg': 0.021, 'neu': 0.941, 'pos': 0.037, 'co...",0.1860,POSITIVE
3,finance,[Bloomberg] New York and California Each Lost ...,mzachi,here's some quotes from the article for those ...,\bGoldman\s+Sachs\b,34,"{'neg': 0.032, 'neu': 0.916, 'pos': 0.051, 'co...",0.6953,POSITIVE
4,finance,Is David Solomon Too Big a Jerk to Run Goldman...,MartianActual,Counterpoint: What other kind of person would ...,\bGoldman\s+Sachs\b,18,"{'neg': 0.179, 'neu': 0.821, 'pos': 0.0, 'comp...",-0.3400,NEGATIVE
5,finance,"Moody's downgrades US banks, warns of possible...",bellayang1216,Based on the information provided:\r \n\r \n...,\bSilicon\s+Valley\s+Bank\b,2,"{'neg': 0.129, 'neu': 0.781, 'pos': 0.09, 'com...",-0.9558,NEGATIVE
6,finance,Why the US is interested in audits of Chinese ...,asuka_rice,"Plenty of lemons everywhere.\n\nCS, SVB and FT...",\bSVB\b,2,"{'neg': 0.129, 'neu': 0.72, 'pos': 0.151, 'com...",0.2249,POSITIVE
7,finance,‘Success fees’ and thirsty emails: inside a $9...,FishFar4370,Musk's desire to have an emotional tantrum and...,\bGoldman\s+Sachs\b,6,"{'neg': 0.038, 'neu': 0.829, 'pos': 0.134, 'co...",0.8981,POSITIVE
8,finance,‘Almost All Loans Are Bad’—Why Banks Aren’t Le...,hcbaron,Extracted article:\n\nBanks would love to lend...,\bBarclays\b,13,"{'neg': 0.049, 'neu': 0.799, 'pos': 0.152, 'co...",0.9977,POSITIVE
9,finance,"Moronic Monday - September 05, 2023 - Your Wee...",14446368,Could we un-pin the SVB thing now?,\bSVB\b,1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,NEUTRAL


In [58]:
df.to_csv('reddit_comments_sentiment.csv', index=False)

## FinBERT

In [2]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone', num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

(…)kust/finbert-tone/resolve/main/vocab.txt: 100%|██████████| 226k/226k [00:00<00:00, 8.84MB/s]


In [16]:
import torch.nn.functional as F

sent_val = []
sent_scores = []
compound_scores = []
X = df['Comment']

for x in X:
    # The tokenizer pads sequences shorter than 512 tokens and truncates longer ones.
    inputs = tokenizer(x, return_tensors='pt', padding=True, truncation=True, max_length=512)
    
    # Getting the model's prediction. The 'outputs' tensor contains logits for each sentiment class.
    outputs = finbert(**inputs)[0]
    
    # Deriving the sentiment label from the prediction by taking the class with the highest logit.
    val = labels[np.argmax(outputs.detach().numpy())]
    
    # Converting the logits to probabilities using the softmax function.
    probabilities = F.softmax(outputs, dim=1).detach().numpy()[0]
    
    # Constructing a dictionary with probabilities for each sentiment class.
    score = {labels[i]: prob for i, prob in enumerate(probabilities)}
    
    # Calculating a compound score as the difference between positive and negative probabilities.
    # This score provides a single metric between -1 and 1.
    compound = probabilities[2] - probabilities[0]

    # Appending the obtained values to their respective lists.
    sent_val.append(val)
    sent_scores.append(score)
    compound_scores.append(compound)

# Adding the sentiment labels, scores, and compound scores to the dataframe as new columns.
df['Sentiment'] = sent_val
df['Sentiment_Scores'] = sent_scores
df['Compound_Score'] = compound_scores

In [17]:
df.head(10)

,Subreddit,Post Title,Comment Author,Comment,Matched Phrase,Upvotes,Sentiment,Sentiment_Scores,Compound_Score
0,finance,‘Almost All Loans Are Bad’—Why Banks Aren’t Le...,hcbaron,Extracted article:\n\nBanks would love to lend...,\bBarclays\b,12,neutral,"{'neutral': 0.8657679, 'positive': 0.030311747...",-0.761848
1,finance,"Moronic Monday - September 05, 2023 - Your Wee...",14446368,Could we un-pin the SVB thing now?,\bSVB\b,1,neutral,"{'neutral': 0.965751, 'positive': 0.0004432395...",-0.931945
2,finance,Bloomberg overhauls management team with Mark ...,marketrent,Per an internal memo sent by founder Mike Bloo...,\bBank\s+of\s+England\b,2,neutral,"{'neutral': 0.9999981, 'positive': 1.5444566e-...",-0.999998
3,finance,[Bloomberg] New York and California Each Lost ...,mzachi,here's some quotes from the article for those ...,\bGoldman\s+Sachs\b,30,negative,"{'neutral': 0.011110356, 'positive': 1.7894672...",0.977761
4,finance,Is David Solomon Too Big a Jerk to Run Goldman...,MartianActual,Counterpoint: What other kind of person would ...,\bGoldman\s+Sachs\b,19,neutral,"{'neutral': 0.9999924, 'positive': 8.0087466e-...",-0.999986
5,finance,"Moody's downgrades US banks, warns of possible...",bellayang1216,Based on the information provided:\r \n\r \n...,\bSilicon\s+Valley\s+Bank\b,2,negative,"{'neutral': 0.0008381561, 'positive': 0.000177...",0.998146
6,finance,Why the US is interested in audits of Chinese ...,asuka_rice,"Plenty of lemons everywhere.\n\nCS, SVB and FT...",\bSVB\b,2,neutral,"{'neutral': 0.7490163, 'positive': 0.052578915...",-0.550612
7,finance,‘Success fees’ and thirsty emails: inside a $9...,FishFar4370,Musk's desire to have an emotional tantrum and...,\bGoldman\s+Sachs\b,6,neutral,"{'neutral': 0.95011884, 'positive': 0.02610915...",-0.926347
8,finance,‘Almost All Loans Are Bad’—Why Banks Aren’t Le...,hcbaron,Extracted article:\n\nBanks would love to lend...,\bBarclays\b,12,neutral,"{'neutral': 0.8657679, 'positive': 0.030311747...",-0.761848
9,finance,"Moronic Monday - September 05, 2023 - Your Wee...",14446368,Could we un-pin the SVB thing now?,\bSVB\b,1,neutral,"{'neutral': 0.965751, 'positive': 0.0004432395...",-0.931945


How to handle differences between 'Sentiment' and 'Compound_Score'?

How to Handle:
* Inspect the Data: Look at a few examples where this inconsistency occurs. This manual inspection can provide insights into why this might be happening.
* Consider Using a Threshold: Instead of relying solely on the compound score's raw value, consider setting a threshold. For instance, only interpret scores below -0.5 as indicative of strong negative sentiment. This can help reduce sensitivity to small variations in score.
* Combine Metrics for Decision Making: If you're making decisions based on the sentiment, consider using both the sent_val and the compound score. For example, only consider a comment as strongly negative if both the sent_val is "negative" and the compound score is below a certain threshold.
* Model Calibration: If the inconsistencies are frequent and problematic, you might want to consider calibrating the model on your specific data or even fine-tuning it.

In conclusion, when interpreting sentiment analysis results, it's crucial to understand that metrics can sometimes provide different perspectives, and each has its strengths and weaknesses. Considering them in tandem and in the context of your specific application will help in achieving more accurate and meaningful interpretations.